<a href="https://colab.research.google.com/github/Barinderkaur15/SellBy_ETL/blob/master/challenege_dataset_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
#for Postgrate driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
# Start a SparkSession
import findspark
findspark.init()
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("challenege").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

--2020-02-04 03:35:23--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.70MB/s    in 0.2s    

2020-02-04 03:35:24 (4.70 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



# **Extract**

In [0]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://barinder-bucket.s3.amazonaws.com/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"), sep="\t", header=True)

In [0]:
#To check our data frame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [0]:
#to count number of rows
df.count()

104975

# **Transform**

In [0]:
#droping NAN vlaues 
df_clean1=df.dropna()
df_clean1.count()

104972

In [0]:
#Deleting dupicate Rows 
df_clean2=df_clean1.distinct()
df_clean2.count()

104972

In [0]:
#checking data type before transformation 
df_clean2.dtypes


[('marketplace', 'string'),
 ('customer_id', 'string'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'string'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'string'),
 ('total_votes', 'string'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [0]:
#Changing the cloumn format
#importing the function 
from pyspark.sql.types import IntegerType,DateType
from pyspark.sql.functions import col

clean_df3 = df_clean2.withColumn("review_date",df_clean2["review_date"].cast(DateType()))
clean_df4 = clean_df3.withColumn("customer_id",clean_df3["customer_id"].cast(IntegerType()))
clean_df5 = clean_df4.withColumn("product_parent",clean_df4["product_parent"].cast(IntegerType()))
clean_df6 = clean_df5.withColumn("star_rating",clean_df5["star_rating"].cast(IntegerType()))
clean_df7= clean_df6.withColumn("helpful_votes",clean_df6["helpful_votes"].cast(IntegerType()))
clean_df_final=clean_df7.withColumn("total_votes",clean_df7["total_votes"].cast(IntegerType()))
clean_df_final.show()







+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47240274|R2JXNXR69I47V2|B004A83PE6|     683281404|6-Pack Mirror Scr...|Mobile_Electronics|          1|            0|          0|   N|                Y|which is a useles...|Didn't notice tha...| 2015-08-19|
|         US|   16990630|R11K27GHAI6195|B00BUFBQWU|     904426906|iPod Touch 5 case...|Mobile_Electronics|      

In [0]:
#creating a dataframe for Postgres
#Table 1 : Review id 
review_id_df=clean_df_final.select(["review_id","customer_id","product_id","product_parent","review_date"])
#review_id check 1 
review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [0]:
#checking Table 1
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2JXNXR69I47V2|   47240274|B004A83PE6|     683281404| 2015-08-19|
|R11K27GHAI6195|   16990630|B00BUFBQWU|     904426906| 2015-08-14|
|R2ALAWL75ID2JM|    2393847|B00SFJBPRK|     385604844| 2015-08-13|
| R8OARX4HEFGK8|   11699577|B00DDM43GO|     188006925| 2015-08-13|
|R28WWNCYL015CU|    1299318|B00J46XO9U|     744008282| 2015-08-10|
| RDC2R9YO70ILG|   29276418|B00TPBG4QE|     880923402| 2015-08-07|
|R1Y60OTOD5GWG0|   24030779|B00MDY3562|      64260162| 2015-07-27|
|R28XVREB47GB9M|   23181159|B00XJCST40|     613294047| 2015-07-25|
|R2HLEUFNMPQN56|   49878060|B00LAG4HN4|     536367292| 2015-07-25|
|R2TBD74G4EZZN5|   35635435|B007Q939ME|     575169462| 2015-07-18|
|R1MOY16ER8CCUA|   41923469|B00SDFHKGG|     262009514| 2015-07-18|
|R1TAG1ZNRY8WZ2|    1804960|B00J46XO9U|     744008282| 2015-07

In [0]:
#creating product table using distinct values only 
from pyspark.sql.functions import col, countDistinct
distinct_value=clean_df_final.distinct()

product_df=distinct_value.select(["product_id","product_title"])
#droping duplicate value 
uniquevalue= product_df.dropDuplicates()

#product_df.select(countDistinct("product_id")).show()

In [0]:
#checking distinct 
uniquevalue.count()




25799

In [0]:
uniquevalue.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00IHS9BPM|Waterproof Blueto...|
|B002ABX29I|Micca Slim Portab...|
|B003NM0DLS|1x4 HDMI Distribu...|
|B00M1Y87I0|      sentey ls 4063|
|B000LXD7D8|Apple iPod nano 2...|
|B00QVXNV60|BRG AH2000G-ANR A...|
|B00L85M8VU|Settonbrothers Ul...|
|B002R1WH2Y|Speck iPod,iPhone...|
|B001I2SJDO|MFX2 Car Power Ch...|
|B00LGNKLXG|Aweek® Car Antenn...|
|B00UN6955M|Aisster(TM) Lot 1...|
|B0056DB4GK|Rode Microphones ...|
|B001B3XNEA|iTrip Auto Univer...|
|B00SUXD5GU|[NEW RELEASE] JEB...|
|B00D6UO8N6|T9680rsame Talkab...|
|B00541H4JK|POWER ACOUSTIK PS...|
|B000BQYM2A|Apple iPod Univer...|
|B000KL2SGS|Stereo Install Da...|
|B006PAZCXY|eForCity® Anti-Gl...|
|B00AE6L022|Patuoxun Remote S...|
+----------+--------------------+
only showing top 20 rows



In [0]:
#To check unique value 
product_df.select(countDistinct("product_id")).show()

+--------------------------+
|count(DISTINCT product_id)|
+--------------------------+
|                     25799|
+--------------------------+



In [0]:
#creating Customer_df
customers=clean_df_final.groupby("customer_id").count()
customers.show()


+-----------+-----+
|customer_id|count|
+-----------+-----+
|   30701952|    1|
|    2446723|    1|
|    6364622|    1|
|   51588676|    2|
|   50328739|    1|
|   45491401|    1|
|    4299027|    1|
|   15579817|    1|
|   11954795|    1|
|     176073|    1|
|   15384487|    1|
|   10521646|    1|
|   12750781|    1|
|   25850881|    1|
|   42182116|    1|
|    8631326|    1|
|   36753682|    1|
|   44842505|    1|
|   36308715|    1|
|   34487450|    1|
+-----------+-----+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import col

customers_df = customers.select(col("customer_id").alias("customer_id"), col("count").alias("customer_count"))
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   30701952|             1|
|    2446723|             1|
|    6364622|             1|
|   51588676|             2|
|   50328739|             1|
|   45491401|             1|
|    4299027|             1|
|   15579817|             1|
|   11954795|             1|
|     176073|             1|
|   15384487|             1|
|   10521646|             1|
|   12750781|             1|
|   25850881|             1|
|   42182116|             1|
|    8631326|             1|
|   36753682|             1|
|   44842505|             1|
|   36308715|             1|
|   34487450|             1|
+-----------+--------------+
only showing top 20 rows



In [0]:
#creating Vine table

vine_df=clean_df_final.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2JXNXR69I47V2|          1|            0|          0|   N|
|R11K27GHAI6195|          1|            0|          0|   N|
|R2ALAWL75ID2JM|          5|            0|          0|   N|
| R8OARX4HEFGK8|          5|            0|          0|   N|
|R28WWNCYL015CU|          5|            0|          0|   N|
| RDC2R9YO70ILG|          5|            0|          0|   N|
|R1Y60OTOD5GWG0|          3|            0|          0|   N|
|R28XVREB47GB9M|          3|            0|          0|   N|
|R2HLEUFNMPQN56|          4|            0|          0|   N|
|R2TBD74G4EZZN5|          5|            2|          2|   N|
|R1MOY16ER8CCUA|          5|            0|          0|   N|
|R1TAG1ZNRY8WZ2|          5|            2|          2|   N|
|R2MQXKIF6W15AW|          1|            0|          0|   N|
|R1CL3BYFZ9IP9Q|          4|            

# **Load**

In [0]:
# Configure settings for RDS
#for Postgrate driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
mode = "append"
jdbc_url="jdbc:postgresql://datavizchallenege.cx9tn8vhsc47.us-east-2.rds.amazonaws.com:5432/postgres"
#hididng password
config = {"user":"postgres",
          "password": "******",
          "driver":"org.postgresql.Driver"}

--2020-02-04 03:39:18--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.60MB/s    in 0.2s    

2020-02-04 03:39:19 (4.60 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [0]:
review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [0]:
review_id_1=review_id_df.dropna()

In [0]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://challengeetl.cx9tn8vhsc47.us-east-2.rds.amazonaws.com:5432/challenge"
config = {"user":"postgres",
          "password": "*******",
          "driver":"org.postgresql.Driver"}

In [0]:
# Write DataFrame to review table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [0]:
# Write DataFrame to products table in RDS
uniquevalue.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [0]:
# Write DataFrame to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [0]:
# Write DataFrame to vine_table table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)